In [36]:
# Load env in local
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
    
print("Loaded OPENAI & LANGSMITH variables")

Loaded OPENAI & LANGSMITH variables


In [37]:
# Load vectordb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

persist_directory = './docs/chroma/'

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print("Loaded existing vector store.")

Loaded existing vector store.


In [29]:
# Load LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [30]:
# Init graph_builder
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)

In [31]:
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

# Tool retrieve define
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vectordb.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của Masterise Homes và bạn sẽ trả lời các câu hỏi của khách hàng về Công ty cũng như các dự án thuộc Công ty."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Khi được hỏi về các thông tin liên quan đến dự án như thời gian bàn giao, tiện ích, vị trí..., hãy tìm trong tài liệu của dự án Masteri Grand View để phản hồi khách hàng"
    )
    response = llm_with_tools.invoke([SystemMessage(system_message_content)] + state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]
    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của Masterise Homes và bạn sẽ trả lời các câu hỏi của khách hàng về Công ty cũng như các dự án thuộc Công ty."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Chỉ dựa trên thông tin trong tài liệu sau, trả lời câu hỏi:"
        "\n\n"
        f"{docs_content}"
        "\n\n"
        "Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự."
        "Nếu thông tin không có trong tài liệu, trả  lời: 'Không tìm thấy thông tin trong tài liệu.', không được sáng tạo nội dung"
        "Nếu có, kèm trích dẫn từ nội dung bên trên"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [32]:
from langgraph.graph import END
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Graph builder
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)
# memory = MemorySaver()
# graph = graph_builder.compile(checkpointer=memory)
graph = graph_builder.compile()

In [35]:
# Execute
config = {"configurable": {"thread_id": "abc123"}}
messages = [
    {"role": "user", "content": "bàn giao"}
]
response = await graph.ainvoke({"messages": messages},config=config)
print(response["messages"][-1].content)

{'recent_tool_messages': [ToolMessage(content="Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}\nContent: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU\nĐƯỢC PHÁT TRIỂN BỞI CÁC \nTÊN TUỔI HÀNG ĐẦU THẾ GIỚI\nTHỜI ĐIỂM VÀNG ĐỂ SỞ HỮU \nNHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE \nGLOBAL CITY\nTHỪA HƯỞNG TOÀN BỘ HỆ TIỆN \nÍCH ĐỈNH CAO CỦA \nTHE GLOBAL CITY\n7.\n8.\nSỞ HỮU LOẠI HÌNH 4PN CỰC KHAN \nHIẾM TRÊN THỊ TRƯỜNG\nMỖI SÀN CHỈ 8CĂN\n\nSource: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}\nContent: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \